# Agent0 Tutorial


Agent0 provides a self contained Hyperdrive simulator using Interactive Local Hyperdrive. It works by managing a local Anvil
instance with an interactive and customizable Hyperdrive deployment, a data collection service, and a dashboard server
showing useful information for analysis.

To start, follow the installation instructions outlined [here](../INSTALL.md) for interactive hyperdrive.


## Simulating trades using Interactive Hyperdrive

The following code initializes a local interactive hyperdrive with
(1) a local chain,
(2) a deployed hyperdrive pool on the local chain, and
(3) a funded agent attached to the pool ready to trade.


In [1]:
import datetime
from fixedpointmath import FixedPoint
from agent0 import LocalHyperdrive, LocalChain

chain = LocalChain()
hyperdrive = LocalHyperdrive(chain)
# Attach the pool to the agent to set as the agent's active pool.
agent = chain.init_agent(base=FixedPoint(100_000), eth=FixedPoint(10), pool=hyperdrive)


24-05-23 14:38:19: INFO: interface.py:88::interface::initialize_engine:
Database interactive_hyperdrive does not exist, creating

24-05-23 14:38:19: WARNING: interface.py:93::interface::initialize_engine:
No postgres connection, retrying

24-05-23 14:38:20: INFO: interface.py:88::interface::initialize_engine:
Database interactive_hyperdrive does not exist, creating


After initializing, we can use the `hyperdrive_agent0` object to simulate trades on the deployed pool.


In [2]:
open_long_event = agent.open_long(base=FixedPoint(100))
open_short_event = agent.open_short(bonds=FixedPoint(100))
add_liquidity_event = agent.add_liquidity(base=FixedPoint(100))

The output of these trades represents the corresponding emitted event from Hyperdrive. For example, the open long event
is shown below. See
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/event_types/index.html) for
documentation on the output event types.


In [3]:
open_long_event

OpenLong(trader='0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25', asset_id=452312848583266388373324160190187140051835877600158453279131187532627765456, maturity_time=1717102800, amount=FixedPoint("100.0"), vault_share_price=FixedPoint("1.00000005707762557"), as_base=True, bond_amount=FixedPoint("100.094931273470295072"), __name__='OpenLong')

We can use Interactive Hyperdrive to simulate advancing time and closing a position. For example, we can simulate
closing the long after a day.


In [4]:
# Advance time for a day
chain.advance_time(datetime.timedelta(days=1))
close_long_event = agent.close_long(
    maturity_time=open_long_event.maturity_time,
    bonds=open_long_event.bond_amount,
)

## Analyzing Hyperdrive

Interactive hyperdrive provides a fully managed data service that collects information from the chain, as
well as providing an interactive dashboard to view information on the pool. Running the dashboard within Interactive
Hyperdrive brings up a webpage that shows you basic information of the trades made on the pool, as well as information
on a specific agent, including the value of the portfolio. You can also view the dashboard directly in your Jupyter notebook.


In [5]:
dashboard = chain.get_dashboard_iframe()
display(dashboard)


  You can now view your Streamlit app in your browser.

  URL: http://localhost:7777

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


For customized analysis, Agent0's simulated environment exposes the underlying data itself. For example, the table
below shows the list of emitted events from the agent.


In [6]:
agent.get_trade_events()

,hyperdrive_address,transaction_hash,block_number,wallet_address,event_type,token_type,maturity_time,token_id,token_delta,base_delta,vault_share_delta,as_base,vault_share_price
0,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,0xc02cdb052e4771b5f2b9a39c0589555751583267f2b1...,22,0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25,OpenLong,LONG,1717102800,LONG-1717102800,100.094931273470295072,-100.000000000000000000,0E-18,True,1.000000057077625570
1,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,0x0c0946148941c6207f1fd5703dfe355f7fe8d11da3a3...,23,0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25,OpenShort,SHORT,1717102800,SHORT-1717102800,100.000000000000000000,-0.097725446539902683,0E-18,True,1.000000076103501845
2,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,0xbf5b5cf1d226167d54a6bd9773065023b1ebb32a269c...,24,0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25,AddLiquidity,LP,None,LP,99.999990485393589134,-100.000000000000000000,0E-18,True,1.000000095129378483
3,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,0x438a65e2b5499c0ba94d91fe931e56dbbf6684010c80...,73,0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25,CloseLong,LONG,1717102800,LONG-1717102800,-100.094931273470295072,100.011780706511699541,0E-18,True,1.000137119495533768


Additionally, we can query the database for the list of open and closed positions, with additional columns
with the unrealized value (i.e., the value of an open position if it was to be closed at this time), the
realized value (i.e., the value in base spent/received from trades on this position), and the PnL (i.e.,
unrealized value + realized value).


In [7]:
agent.get_positions(show_closed_positions=True)

,hyperdrive_name,hyperdrive_address,block_number,username,wallet_address,token_type,maturity_time,token_id,token_balance,unrealized_value,realized_value,pnl,last_balance_update_block
0,delv_7_day,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,73,0x3903...FA25,0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25,LONG,1717102800,LONG-1717102800,0E-18,0E-18,0.011780706511699541,0.011780706511699541,73
1,delv_7_day,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,73,0x3903...FA25,0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25,LP,None,LP,99.999990485393589134,100.013702436088523469,-100.000000000000000000,0.013702436088523469,24
2,delv_7_day,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,73,0x3903...FA25,0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25,SHORT,1717102800,SHORT-1717102800,100.000000000000000000,0.095815066127594798,-0.097725446539902683,-0.001910380412307885,23


See the `get_*` functions defined in
interactive hyperdrive
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/interactive_hyperdrive/index.html)
for more information.


## Trading policies

Agent0 introduces agent policies, which allows an agent to make trades based on some defined behavior.
Agent0 provides a set of ready-to-use policies. For example, the `random` policy simply makes random trades
on the pool.


In [8]:
from agent0 import PolicyZoo

print(PolicyZoo.random.description())

  A simple demonstration agent that chooses its actions randomly.
  It can take 7 actions: open/close longs and shorts, add/remove liquidity, and redeem withdraw shares.
  Trade size is randomly drawn from a normal distribution with mean of 10% of budget and standard deviation of 1% of budget.
  A close action picks a random open position of the given type (long or short) and attempts to close its entire size.
  Withdrawals of liquidity and redemption of withdrawal shares is sized the same as an open position: N(0.1, 0.01) * budget.


We can set a policy for an agent and execute the policy using interactive hyperdrive.
The output is then a list of output events corresponding with the list of actions the
policy took in that step. In the example below, we set the random policy to the previously
created agent and execute one iteration of the policy, then print out the trade the agent made.


In [9]:
agent.set_active(policy=PolicyZoo.random, policy_config=PolicyZoo.random.Config(rng_seed=123))
agent.execute_policy_action()

[AddLiquidity(provider='0x390362b0Ba14d281A2172bE93d3d1eEd9b5bFA25', lp_amount=FixedPoint("11275.081386354898775218"), amount=FixedPoint("11276.62763464994062336"), vault_share_price=FixedPoint("1.000137138524017776"), as_base=True, lp_share_price=FixedPoint("1.000000000024439597"), __name__='AddLiquidity')]

See
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/policies/index.html#)
for a list and description of all implemented policies.


### Custom Policies

Agent0 also provides an interface to write a custom policy. In this tutorial, we'll write a
policy that simply opens a long if the fixed rate is above a threshold.
Similarly to existing policies, this policy can then be attached to interactive hyperdrive to
simulate the policy within the simulator.

We'll start by defining the class that subclasses from the base policy and define a custom policy
configuration that defines the threshold. We need to overwrite the `action` method in the
policy, which defines what actions the policy takes. In the example below, the agent will simply
open a single long for `open_long_amount` when the fixed rate reaches the `fixed_rate_threshold`,
and close the long at maturity.


In [10]:
# Relevant imports
from agent0 import HyperdriveBasePolicy, open_long_trade, close_long_trade
from dataclasses import dataclass

In [11]:
class OpenLongPolicy(HyperdriveBasePolicy):
    @dataclass(kw_only=True)
    class Config(HyperdriveBasePolicy.Config):
        fixed_rate_threshold: FixedPoint
        open_long_amount: FixedPoint

    def action(self, interface, wallet):
        # Defines if the bot is done trading. We expect this bot to run continuously,
        # so this is always false.
        done_trading = False
        # If no longs in wallet, we check our fixed rate threshold and open the long if threshold reached.
        if len(wallet.longs) == 0:
            if interface.calc_spot_rate() > self.config.fixed_rate_threshold:
                return [open_long_trade(self.config.open_long_amount)], done_trading
        # If there are longs in the wallet, we check for maturity and close them if maturity reached.
        else:
            for maturity_time, long in wallet.longs.items():
                if interface.get_block_timestamp(interface.get_current_block()) >= maturity_time:
                    return [close_long_trade(long.balance, maturity_time)], done_trading
        # We don't do any trades otherwise.
        return [], done_trading

See [here](custom_policy.py) for a more in depth example of writing a policy.


### Running the trading policy in Interactive Hyperdrive

We can test our new policy using interactive hyperdrive before running it on an actual chain. We can do this by
either setting a new active policy, or initializing a new agent while setting the policy during initialization.
We'll do the latter here. Note that we pass in the fixed rate threshold of 6% as a parameter to the policy.


In [12]:
policy_agent = chain.init_agent(
    base=FixedPoint(1_000_000),
    eth=FixedPoint(10),
    pool=hyperdrive,
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint(100_000),
    ),
)

Lets take a look at the fixed rate after the set of trades we've made. We can see below that the current fixed rate sits
right around 5%.


In [13]:
hyperdrive.get_pool_info()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,19,2024-05-23 21:42:07,0.050000000000000000
1,20,2024-05-23 21:42:19,0.050000000000000000
2,21,2024-05-23 21:42:31,0.050000000000000000
3,22,2024-05-23 21:42:43,0.049999757141356994
4,23,2024-05-23 21:42:55,0.049999999766702777
5,24,2024-05-23 21:43:07,0.049999999766702777
6,72,2024-05-24 21:43:19,0.049999999766702777
7,73,2024-05-24 21:43:31,0.050000207907633608
8,74,2024-05-24 21:43:43,0.050000207907633608


We can now execute our new agent's policy. Since the fixed rate is below the threshold, we expect the agent to not make
any trades, denoted by the output of `execute_policy_action` being an empty list.


In [14]:
policy_agent.execute_policy_action()

[]

Let's make a trade to push the fixed rate to be above 6%.


In [15]:
policy_agent.open_short(bonds=FixedPoint(20_000_000))
hyperdrive.get_pool_info()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,19,2024-05-23 21:42:07,0.050000000000000000
1,20,2024-05-23 21:42:19,0.050000000000000000
2,21,2024-05-23 21:42:31,0.050000000000000000
3,22,2024-05-23 21:42:43,0.049999757141356994
4,23,2024-05-23 21:42:55,0.049999999766702777
5,24,2024-05-23 21:43:07,0.049999999766702777
6,72,2024-05-24 21:43:19,0.049999999766702777
7,73,2024-05-24 21:43:31,0.050000207907633608
8,74,2024-05-24 21:43:43,0.050000207907633608
9,75,2024-05-24 21:43:55,0.050000207907633608


Once again, let's execute our policy and view the output. We can see that our policy opened a new long because the
fixed rate is above 6%.


In [16]:
policy_agent.execute_policy_action()

[OpenLong(trader='0x4Ad575d8e2Cac1D652296558b5582073B776592e', asset_id=452312848583266388373324160190187140051835877600158453279131187532627851856, maturity_time=1717189200, amount=FixedPoint("100000.0"), vault_share_price=FixedPoint("1.000137214637956344"), as_base=True, bond_amount=FixedPoint("100252.699002323955738645"), __name__='OpenLong')]

## Connecting to a remote chain

Along with a fully managed simulator, interactive hyperdrive also provides an interface for connecting to any existing remote chain and deployed hyperdrive pool. We can use this interface to make trades or execute any policy on the deployed hyperdrive pool.

In this tutorial, we'll use the Anvil node and Hyperdrive pool being hosted by the `LocalChain`
and `LocalHyperdrive` objects, but we can simply replace the corresponding configurations with any valid RPC and contract addresses.


In [17]:
from agent0 import Chain, Hyperdrive

# Get the RPC address and the hyperdrive contract addresses to connect to from the managed
# interactive hyperdrive objects.
rpc_uri = chain.rpc_uri
hyperdrive_address = hyperdrive.hyperdrive_address

# We can specify other parameters as such.
# rpc_uri = "<rpc_uri>"
# hyperdrive_address = "0x<hyperdrive_address>"

# Connect to the remote chain and hyperdrive objects.
# We need to set a different db port here to avoid port collisions with the local chain.
remote_chain = Chain(rpc_uri, Chain.Config(db_port=1234))
remote_hyperdrive = Hyperdrive(remote_chain, hyperdrive_address)


24-05-23 14:38:26: INFO: interface.py:88::interface::initialize_engine:
Database interactive_hyperdrive does not exist, creating

24-05-23 14:38:26: WARNING: interface.py:93::interface::initialize_engine:
No postgres connection, retrying

24-05-23 14:38:27: INFO: interface.py:88::interface::initialize_engine:
Database interactive_hyperdrive does not exist, creating


We now need to initialize an agent by providing a private key to a wallet. In this tutorial,
we'll simply use a random private key to initialize a new wallet, then fund the agent using
the provided `add_funds` method.

<div class="alert alert-block alert-info"> 
NOTE:: The <code>add_funds</code> method calls the <code>anvil_setBalance</code> to fund Eth and 
<code>mint</code> on the base token contract to fund the wallet, which is only possible due to the 
remote chain and base token contract supporting such calls. It's up to the user to ensure the wallet 
associated with the private key is funded when running on a non-test chain.
</div>

<div class="alert alert-block alert-danger"> 
WARNING:: This tutorial generates a random private key and funds this wallet. In practice, the private
key should be passed into the script via an environment variable secret, and one should limit access
to the generated agent object to only the owner of the private key.
</div>

We also attach our custom agent as well to allow the agent to execute the policy we wrote earlier.


In [18]:
from agent0.core.base.make_key import make_private_key

remote_agent = remote_chain.init_agent(
    private_key=make_private_key(),
    pool=remote_hyperdrive,
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint(100_000),
    ),
)
# Add funds to the agent
remote_agent.add_funds(base=FixedPoint(1_000_000), eth=FixedPoint(100))
# Give the hyperdrive contract max approval to withdrawal funds from the remote agent
# TODO need to add setting a custom approval amount
remote_agent.set_max_approval()

This agent is now able to make trades on the remote chain, as well as executing the underlying policy


In [19]:
remote_agent.open_short(bonds=FixedPoint(200))

OpenShort(trader='0xe5fa0048f45D461b7656bB109869a9B4bE300Ddd', asset_id=904625697166532776746648320380374280103671755200316906558262375063538514512, maturity_time=1717189200, amount=FixedPoint("0.514471971450155471"), vault_share_price=FixedPoint("1.0001372717234138"), as_base=True, base_proceeds=FixedPoint("199.487485351529177335"), bond_amount=FixedPoint("200.0"), __name__='OpenShort')

In [20]:
remote_agent.execute_policy_action()

[OpenLong(trader='0xe5fa0048f45D461b7656bB109869a9B4bE300Ddd', asset_id=452312848583266388373324160190187140051835877600158453279131187532627851856, maturity_time=1717189200, amount=FixedPoint("100000.0"), vault_share_price=FixedPoint("1.000137290751900705"), as_base=True, bond_amount=FixedPoint("100250.937543966939989084"), __name__='OpenLong')]

We find that the wallet contains both the short we opened manually and the long we opened using
the policy.


In [21]:
remote_agent.get_positions(pool_filter=remote_hyperdrive)

,hyperdrive_name,hyperdrive_address,block_number,username,wallet_address,token_type,maturity_time,token_id,token_balance,unrealized_value,realized_value,pnl,last_balance_update_block
0,delv_7_day,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,82,0xe5fa...0Ddd,0xe5fa0048f45D461b7656bB109869a9B4bE300Ddd,LONG,1717189200,LONG-1717189200,100250.937543966939989084,99981.210313844005556068,-100000.000000000000000000,-18.789686155994443932,82
1,delv_7_day,0xA927d75e96C0176c50d86AeBF2956710c228A1C0,82,0xe5fa...0Ddd,0xe5fa0048f45D461b7656bB109869a9B4bE300Ddd,SHORT,1717189200,SHORT-1717189200,200.000000000000000000,0.500777639688480618,-0.514471971450155471,-0.013694331761674853,81


## Cleanup

Recall that the local chain created at the beginning of this tutorial produces resources under
the hood that need to be cleaned up. The call below releases all the resources used by
interactive hyperdrive.

**Note**: This cell will terminate process for the database, local chain, and the dashboard running above.


In [22]:
chain.cleanup()

  Stopping...


## Final words

In conclusion, this tutorial showcases the interactive hyperdrive workflow. We first explored
the simulation Hyperdrive environment and the analysis tools it provides. We then explored
trading policies and wrote a quick example of a custom policy, and how they plug into the
interactive hyperdrive ecosystem. Finally, we showed how to execute trades and policies
on any remote hyperdrive deployment.

Happy trading!
